In [ ]:
import pandas as pd

In [ ]:
log_path = 'data_format1/user_log_format1.csv'
info_path = 'data_format1/user_info_format1.csv'

In [ ]:
log_df = pd.read_csv(log_path)

In [ ]:
log_df.dropna(inplace=True)
log_df['brand_id'] = log_df['brand_id'].astype('int')
log_df = log_df.groupby(by=['user_id', 'item_id', 'cat_id', 'seller_id', 'brand_id'], as_index=False)['action_type'].agg(list)

In [ ]:
log_df['purchase'] = log_df['action_type'].map(lambda x: int(2 in x))
log_df['favourite'] = log_df['action_type'].map(lambda x: int(3 in x))
log_df.drop('action_type', axis=1, inplace=True)

In [ ]:
user_df = pd.read_csv(info_path)
user_df.dropna(inplace=True)
user_df['age_range'] = user_df['age_range'].astype('int')
user_df['gender'] = user_df['gender'].astype('int')

In [ ]:
data_df = pd.merge(log_df, user_df, how='left', on='user_id')

In [ ]:
data_df.dropna(inplace=True)

In [ ]:
data_df

In [ ]:
def filte(df: pd.DataFrame, field_name, target_field='purchase', thresh=5):
    items = set()
    df = df.groupby(by=field_name, as_index=False)[target_field].agg(sum)
    for k, v in df.values:
        if v < thresh:
            items.add(k)

    return items

In [ ]:
ban_users = filte(data_df, 'user_id', thresh=10)
ban_items = filte(data_df, 'item_id', thresh=10)

In [ ]:
drop_user = data_df['user_id'].isin(ban_users)
data_df.drop(data_df.index[drop_user], inplace=True)

In [ ]:
drop_item = data_df['item_id'].isin(ban_items)
data_df.drop(data_df.index[drop_item], inplace=True)

In [ ]:
fields = {
    'user_id': 'user_id:token',
    'item_id': 'item_id:token',
    'cat_id': 'cat_id:token',
    'seller_id': 'seller_id:token',
    'brand_id': 'brand_id:token',
    'age_range': 'age_range:token',
    'gender': 'gender:token',
    'purchase': 'purchase:label',
    'favourite': 'favourite:label'
}
data_df.rename(columns=fields, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
lbe = LabelEncoder()
data_df["user_id:token"] = lbe.fit_transform(data_df["user_id:token"]) + 1
data_df["item_id:token"] = lbe.fit_transform(data_df["item_id:token"]) + 1
data_df["cat_id:token"] = lbe.fit_transform(data_df["cat_id:token"]) + 1
data_df["seller_id:token"] = lbe.fit_transform(data_df["seller_id:token"]) + 1
data_df["brand_id:token"] = lbe.fit_transform(data_df["brand_id:token"]) + 1

In [ ]:
df_train, df_val = train_test_split(data_df, test_size=0.2)
df_val, df_test = train_test_split(df_val, test_size=0.5)

df_train.to_csv("ijcai15_train.csv", index=False)
df_val.to_csv("ijcai15_val.csv", index=False)
df_test.to_csv("ijcai15_test.csv", index=False)

In [ ]:
data_df.to_csv('ijcai_15.inter', index=False)